# Assignment 3: Regression & Ensembles

In [1]:
# enter your name and UFL email address
name = 'enter your name'
email = 'enter your email'

name = 'solution' # ###- 
email = 'solution' # ###- 

In [2]:
if name == 'enter your name' or email == 'enter your email':
    assert False, 'Enter your name & email first!'
else:
    print('Assignment 3 -- name: {}, email: {}\n'.format(name, email))
    
    # Load packages we need
    import sys
    import os
    import time

    import numpy as np
    import pandas as pd
    import sklearn

    from matplotlib import pyplot as plt
    plt.rcParams.update({'font.size': 16})

    # Let's check our software versions
    print('### Python version: ' + __import__('sys').version)
    print('### NumPy version: ' + np.__version__)
    print('### Scikit-learn version: ' + sklearn.__version__)
    print('------------')


    # load our packages / code
    sys.path.insert(1, '../common/')
    import utils
    import plots

Assignment 3 -- name: solution, email: solution

### Python version: 3.8.5 (default, Jan 27 2021, 15:41:15) 
[GCC 9.3.0]
### NumPy version: 1.19.5
### Scikit-learn version: 0.24.0
------------


In [3]:
# global parameters to control behavior of the pre-processing, ML, analysis, etc.
seed = 42

# deterministic seed for reproducibility
##rng = np.random.default_rng(seed)  # best practice but not fully implemented in scikit-learn
np.random.seed(seed)

prop_vec = [14, 3, 3]

## Part 1: Loading and Pre-processing Data

### For this assignment we'll load the Bike Sharing dataset (hourly)
### This dataset contains features of users bike sharing/rental on an hourly basis.
### The task is to predict how many users are sharing/renting a bike.

### Loading data

In [4]:
### Note: this dataset has missing values (artificially introduced), which you'll need to fill in before you can train a model
df = pd.read_csv('../data/bikesharehour.csv.gz', compression='gzip', header=0, na_values='?')

# Check that we loaded the data as expected
df_expected_shape = (17379, 15)

assert df.shape == df_expected_shape, 'Unexpected shape of df!'

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      16320 non-null  float64
 1   year        16231 non-null  float64
 2   month       16304 non-null  float64
 3   hour        16254 non-null  float64
 4   holiday     16277 non-null  float64
 5   weekday     16282 non-null  float64
 6   workingday  16297 non-null  float64
 7   weathersit  16324 non-null  float64
 8   temp        16242 non-null  float64
 9   atemp       16271 non-null  float64
 10  hum         16252 non-null  float64
 11  windspeed   16281 non-null  float64
 12  registered  16244 non-null  float64
 13  nsqrtc      16263 non-null  float64
 14  count       17379 non-null  int64  
dtypes: float64(14), int64(1)
memory usage: 2.0 MB


In [5]:
## what does the data look like?
df.head()

,season,year,month,hour,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,registered,nsqrtc,count
0,1.0,0.0,NaN,0.0,0.0,6.0,0.0,1.0,NaN,0.0,0.0,0.0,13.0,-5.0,16
1,1.0,0.0,NaN,1.0,0.0,6.0,0.0,1.0,NaN,0.0,0.0,0.0,32.0,-8.0,40
2,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,27.0,-7.0,32
3,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,-5.0,13
4,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1


### There are some NaNs which we'll have to impute!

In [6]:
# grab all the data as a numpy matrix
all_xy = df.to_numpy()

col_names = [c for c in df.columns]
features = col_names[:-1]
target = col_names[-1]

In [7]:
print('features: {} --- target: {}'.format(features, target))

features: ['season', 'year', 'month', 'hour', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'registered', 'nsqrtc'] --- target: count


In [8]:
# how many NaNs in each column?
np.sum(np.isnan(all_xy), axis=0)

array([1059, 1148, 1075, 1125, 1102, 1097, 1082, 1055, 1137, 1108, 1127,
       1098, 1135, 1116,    0])

### Observe: no NaNs in the target/value column
### About 1000+ NaNs in each feature

In [9]:
# split into x and y
all_x_nan = all_xy[:,:-1]
all_y = all_xy[:,-1]

## [Task 1] (10 points) Let's impute the missing values! Use Scikit-learn's SimpleImputer to replace all NaNs in 'all_x_nan' with the *most frequent* value in each column. Use copy=True and store the results in 'all_x' 

In [10]:
from sklearn.impute import SimpleImputer

###* put your code here (~2-3 lines) *###
mf_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent', copy=True)

all_x_mf = mf_imputer.fit_transform(all_x_nan)
all_x = all_x_mf

In [11]:
# check that the shape is correct
assert all_x.shape == (17379, 14)

# check that there are no more NaNs
assert np.sum(np.sum(np.isnan(all_x), axis=0)) == 0

### Rescale the features

In [12]:
# We'll min-max normalize the features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(copy=True)
scaler.fit(all_x) 

scaled_all_x = scaler.transform(all_x)

### Let's split the data

In [13]:
# split the data into train, test, val
train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(scaled_all_x, all_y, prop_vec, shuffle=True, seed=seed)

# sanity check shapes
train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape

((12166, 14), (12166,), (2607, 14), (2607,), (2606, 14), (2606,))

## [Task 2] (30 points) Let's train linear models!

### [Task 2a] (2 points) Train a Linear Regression model using the default hyperparameters. 

In [14]:
from sklearn.linear_model import LinearRegression

### Train a linear regression model on the training data (train_x, train_y)
### Call the resulting trained model 'lrmodel'
###* put your code here (~1 line) *###
lrmodel = LinearRegression().fit(train_x, train_y)

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def r2_mse_mae_eval(model, pref=''):
    # R^2 the coefficient of determination
    r2_train = model.score(train_x, train_y)
    r2_val = model.score(val_x, val_y)

    print('{}Train R^2: {:.3f}, Val  R^2: {:.3f}'.format(pref, r2_train, r2_val))

    train_pred = model.predict(train_x)
    val_pred = model.predict(val_x)

    # measure the error (MSE) wrt true target
    train_error = mean_squared_error(train_pred, train_y)
    val_error = mean_squared_error(val_pred, val_y)

    print('{}Train MSE: {:.3f}, Val MSE: {:.3f}'.format(pref, train_error, val_error))
    
    train_error = mean_absolute_error(train_pred, train_y)
    val_error = mean_absolute_error(val_pred, val_y)

    print('{}Train MAE: {:.3f}, Val MAE: {:.3f}'.format(pref, train_error, val_error))
    
r2_mse_mae_eval(lrmodel)

Train R^2: 0.847, Val  R^2: 0.833
Train MSE: 5033.098, Val MSE: 5532.778
Train MAE: 39.039, Val MAE: 39.742


### [Task 2b] (3 points) How good is that model? (A few sentences is fine.)

In [16]:
###* put your answer as comment here *###
#
#
# It's definitely a good model judging by the R^2 score. It's not perfect, of course.
# Note: it's hard to say precisely how good without some kind of baseline... 
#

### Let's setup some functions so we can tune hyperparameters

In [17]:
## some code to do a grid search and automatically train & evaluate the model with the best hyperparams.
from sklearn.model_selection import GridSearchCV

def do_grid_search(model, param_grid, x, y):
    gs = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error')
    gs_res = gs.fit(x, y)
    return  gs_res.best_params_


def search_train_eval(model, param_grid, tr_x=train_x, tr_y=train_y, v_x=val_x, v_y=val_y):
    
    # since we do CV for the grid search, let's concatenate the train and val sets for it
    search_x = np.r_[tr_x, v_x]
    search_y = np.r_[tr_y, v_y]
    
    hyperparams = do_grid_search(model, param_grid, search_x, search_y)
    
    class_obj = type(model)
    m = class_obj(**hyperparams).fit(tr_x, tr_y)
    
    cn = str(class_obj).split("'")[1]
    cn = cn.split('.')[-1]
    print('{}({})'.format(cn, hyperparams))

    r2_mse_mae_eval(m, pref='\t')

    return m

### [Task 2c] (5 points) Do a grid search to tune hyperparameters and train an ElasticNet model. You can choose the values of hyperparameters your search over, but you must search over 'alpha' and 'l1_ratio'. Ensure that during the search the training of models converges in all cases (you should set max_iter or change your chosen values). 

In [18]:
### Hint: you should define a parameter grid dictionary and call search_train_eval() to do the actual search
###* put your code here (~3 lines) *###
from sklearn.linear_model import ElasticNet

param_grid = {'alpha':[0.01, 0.1, 1.0, 10.0], 'l1_ratio': [0.1, 0.5, 1.0]}
enmodel = search_train_eval(ElasticNet(max_iter=1000), param_grid)

ElasticNet({'alpha': 0.01, 'l1_ratio': 1.0})
	Train R^2: 0.847, Val  R^2: 0.833
	Train MSE: 5033.159, Val MSE: 5531.757
	Train MAE: 39.032, Val MAE: 39.729


### [Task 2d] (2 points) Do a grid search to tune hyperparameters and train a Ridge Regression model. You can choose the values of hyperparameters your search over, but you must search over 'alpha'. Ensure that during the search the training of models converges in all cases (you should set max_iter or change your chosen values). 

In [19]:
###* put your code here (~3 lines) *###
from sklearn.linear_model import Ridge

param_grid = {'alpha':[0.01, 0.1, 1.0, 10.0]}
rmodel = search_train_eval(Ridge(max_iter=1000), param_grid)

Ridge({'alpha': 0.1})
	Train R^2: 0.847, Val  R^2: 0.833
	Train MSE: 5033.101, Val MSE: 5532.577
	Train MAE: 39.046, Val MAE: 39.749


### [Task 2e] (3 points) Print the parameter values (w and b) of the ElasticNet and Ridge Regression models.

In [20]:
# Print the weights and bias for both models
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### Make sure you print the weights and bias for both models and that it is clear which is which.
###* put your code here (~2 lines) *###
print('ElasticNet model -- w: {}, b: {:.3f}'.format(enmodel.coef_, enmodel.intercept_))
print('Ridge model -- w: {}, b: {:.3f}'.format(rmodel.coef_, rmodel.intercept_))

ElasticNet model -- w: [ 26.043  14.238 -10.552  52.773 -7.933  6.554 -30.398 -30.660  0.000
 -0.000 -12.625  28.327  926.905 -14.186], b: 20.531
Ridge model -- w: [ 26.431  14.271 -10.972  52.865 -8.324  6.631 -30.472 -30.828  1.476
 -16.757 -13.215  28.873  926.780 -14.613], b: 20.661


### [Task 2f] (2 points) How similar are the parameter values of the two models?

In [21]:
###* put your answer as comment here *###
#
# Parameters are similar in some ways but different in other ways. The intercept is similar, 
# but the weights of some features are quite different (since the models perform similarly, it 
# suggests some features may be correlated to others)
#
#
absv = np.abs(enmodel.coef_ - rmodel.coef_)
absv, features[np.argmax(absv)]


(array([ 0.388,  0.033,  0.420,  0.092,  0.391,  0.077,  0.074,  0.168,
         1.476,  16.757,  0.590,  0.546,  0.125,  0.427]),
 'atemp')

### [Task 2g] (8 points) For each of the two models, display the three most important features alongside with their coefficients. Are these the same across both models?
### What are the coefficients? Which feature is the most important?

In [22]:
###* put your code here *###
feat_idx_elasticnet = np.flip(np.argsort(np.abs(enmodel.coef_)))
feat_idx_ridge = np.flip(np.argsort(np.abs(rmodel.coef_)))

for i in range(0, 3):
    enidx = feat_idx_elasticnet[i]
    print('[ElasticNet] Feature {} ({}) has weight: {:.3f}'.format(enidx, features[enidx], enmodel.coef_[enidx]))
    ridx = feat_idx_ridge[i]
    print('[Ridge] Feature {} ({}) has weight: {:.3f}'.format(ridx, features[ridx], rmodel.coef_[ridx]))
    print()


###* put your answer as comment here *###
# 
# ElasticNet: registered (923.807), hour (53.326), weathersit (-31.456)
# Ridge: registered (923.725), hour (53.412), weathersit (-31.623)
#
# The two models are very similar
#

[ElasticNet] Feature 12 (registered) has weight: 926.905
[Ridge] Feature 12 (registered) has weight: 926.780

[ElasticNet] Feature 3 (hour) has weight: 52.773
[Ridge] Feature 3 (hour) has weight: 52.865

[ElasticNet] Feature 7 (weathersit) has weight: -30.660
[Ridge] Feature 7 (weathersit) has weight: -30.828



### [Task 2h] (5 points) Take a look at the code of search_train_eval() and do_grid_search(). Answer the following questions: 
### 1. Why is the scoring function for the grid search 'neg_mean_squared_error' (as opposed to 'mean_squared_error')? 
### 2. Why is it okay to do the search over search_x and search_y which are the concatenation of the train and 'validation sets?

In [23]:
### Hint: take a look at the documentation of scikit-learn for GridSearchCV and related classes.
###* put your answer as comment here *###
#
# 1. GridSearchCV (like other hyperparam tuning of scikit-learn) is set up to pick the hyperparam combination
# that maximizes a score function. So if we want to minimize MSE, we can equivalently maximize -MSE.
#
# 2.GridSearchCV is doing cross-validation and averaging results. So it's okay if we combine train and val because
# it will not overfit the search on the validation set (due to cross-val & averaging). 
#

## [Task 3] (30 points) Let's train polynomial regression models!

### [Task 3a] (5 points) Use PolynomialFeatures to create a version of the data with all features of degree 2.

In [24]:
from sklearn.preprocessing import PolynomialFeatures

### Use PolynomialFeatures to create a version of the data with all features of degree 2. Make sure to allow interactions (interaction_only=False) and set include_bias=False.
### Store the result in 'all_x_polyf'. Ensure that you make a copy of the original data and you use the scaled features ('scaled_all_x')!
###* put your code here (~2 lines) *###
polyf = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
all_x_polyf = polyf.fit_transform(scaled_all_x.copy())

assert all_x_polyf.shape == (17379, 119)

# split the data into train, test, val
train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(all_x_polyf, all_y, prop_vec, shuffle=True, seed=seed)

### Let's train a LinearRegression model and a Ridge model on our polynomial features.

In [25]:
# Train a linear regression model
pf_lrmodel = LinearRegression().fit(train_x, train_y)
r2_mse_mae_eval(pf_lrmodel)

print()

# Train a Ridge regression model
pf_ridgemodel = Ridge(alpha=0.5).fit(train_x, train_y)
r2_mse_mae_eval(pf_ridgemodel)

Train R^2: 0.927, Val  R^2: 0.923
Train MSE: 2412.444, Val MSE: 2553.487
Train MAE: 24.959, Val MAE: 24.840

Train R^2: 0.926, Val  R^2: 0.921
Train MSE: 2444.731, Val MSE: 2634.107
Train MAE: 24.644, Val MAE: 24.661


### [Task 3b] (5 points) What is the difference between LinearRegression and Ridge? (A sentence or two is fine.)

In [26]:
###* put your answer as comment here *###
#
# Ridge will do L2-regularization whereas LinearRegression will do not any regularization (OLS).
#
#

### [Task 3c] (5 points) Look at (e.g., print) the parameters of both the LinearRegression model ('pf_lrmodel') and the Ridge model ('pf_ridgemodel'). What do you notice? Explain what is going on.

In [27]:
###* put your code here *###
print('LinearRegression model (poly. f.) -- w: {}, b: {:.3f}'.format(pf_lrmodel.coef_, pf_lrmodel.intercept_))
print()
print('Ridge model (poly. f.) -- w: {}, b: {:.3f}'.format(pf_ridgemodel.coef_, pf_ridgemodel.intercept_))

###  What do you notice? Explain what is going on.
###* put your answer as comment here *###
#
# Values of the parameters/coefficients for the LinearRegression models are all over the place (some have very large magnitude both neg. and pos.). 
# This is because the model has no regularizing constraints! In contrast, coefficients of the Ridge model seem more reasonable.
# Note: the regularization constant (alpha) for Ridge is not particularly large which means the regularization effect is small
# yet this has a significant effect on the coefficients!
#

LinearRegression model (poly. f.) -- w: [ 5.684  153343876892390.719  41.607  123.876 -54293547848441.305  26.913
  1878227560857.761  34.037 -14021413734269.637 -4597994788482.195
  595998891306.100 -23.689  721.223 -1788.284 -16.017  13.763  25.116
  43.184  1.287 -5.791  0.372 -11.271 -9480688549146.117
 -3966266098589.549  1.123  11.148 -83.558 -18.907 -153343876892383.438
 -11.635  40.681 -6.398  1.560  5.211 -10.750  6301087549983.425
 -1005791667980.723 -0.895  23.811 -108.793 -16.492 -40.672 -5.789 -4.735
  5.906 -1.596  6.668  3219927304071.929 -559213520135.313 -0.695 -5.703
  1.354  1.609 -110.365 -0.508  9.141  28.551 -46.773  3742573190253.013
 -574688232509.478 -17.676  47.791 -230.324 -10.719  54293547848427.492
 -7.742 -18.162 -5.323 -119809244824.540 -211884697337.347
 -99807676006.857  42.522 -6.065  20.273 -25.729  3.786  0.656
  6710637521339.792  967483716330.801 -3.277 -30.202 -23.461  9.766
 -1878227560865.586 -10.750 -49666834501.250  2705499065414.198 -7.961
  

### [Task 3d] (5 points) Focus on the Ridge model. What are the three most important features? 

In [28]:
### Print the three most important features alongside with their weights.
### Hint: you can use get_feature_names() method of PolynomialFeatures to relate polynomial features to the original features.
###* put your code here *###

poly_features = polyf.get_feature_names(features)
feat_idx = np.flip(np.argsort(np.abs(pf_ridgemodel.coef_)))

for i in range(0, 3):
    print('Feature {} ({}) has weight: {:.3f}'.format(feat_idx[i], poly_features[feat_idx[i]], pf_ridgemodel.coef_[feat_idx[i]]))

Feature 118 (nsqrtc^2) has weight: 1513.202
Feature 13 (nsqrtc) has weight: -1469.418
Feature 12 (registered) has weight: 717.231


### [Task 3e] (5 points) Let's use only these three most important features. Extract the three features from the polynomial features to create a new feature matrix with three columns.

In [29]:
### Extract the three features from 'all_x_polyf' and store the results in 'all_x_3most'
###* put your code here *###

all_x_3most = all_x_polyf[:, [feat_idx[0], feat_idx[1], feat_idx[2]]]

assert all_x_3most.shape == (17379, 3)

# split the data into train, test, val
train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(all_x_3most, all_y, prop_vec, shuffle=True, seed=seed)

### [Task 3f] (2 points) Now train a LinearRegression model (default hyperparams) on the training data from 'all_x_3most'. What do you observe about the performance of this model? What is your conclusion?

In [30]:
###* put your code here *###
threemost_model = LinearRegression().fit(train_x, train_y)
r2_mse_mae_eval(threemost_model)

Train R^2: 0.912, Val  R^2: 0.904
Train MSE: 2903.069, Val MSE: 3181.395
Train MAE: 23.434, Val MAE: 23.624


### [Task 3g] (3 points) How good is that model? What do you conclude?

In [31]:
###* put your answer as comment here *###
#
# We observe that the model is not quite as good. However, it's performance is not that far off. 
# Considering that this model only relies on three features and thus is less likely to overfit, it is a reasonable alternative. 
# because of the three features is a polynomial feature, we also conclude that the relationship in the data 
# between features and target is somewhat non-linear.
#

## [Task 4] (30 points) Trees, More Trees, lots of Trees!

### We need to reset the data to the original form (before polynomial features)

In [32]:
# let's do some cleanup
del train_x, train_y, test_x, test_y, val_x, val_y

# split the data into train, test, val
train_x, train_y, test_x, test_y, val_x, val_y = utils.train_test_val_split(scaled_all_x, all_y, prop_vec, shuffle=True, seed=seed)

# sanity check shapes
train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape
assert train_x.shape == (12166, 14)

### Let's train a decision tree!

In [33]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

dtmodel = DecisionTreeRegressor(random_state=seed).fit(train_x, train_y)

### Uncomment the code in the cell below if you have some time to wait around and want to visualize the tree, otherwise skip it!

In [34]:
### This will take a long time (10-20 minutes); skip if you are in a hurry
# let's plot what the tree looks like
#plt.figure(figsize=(16,12))
#
#plot_tree(dtmodel, feature_names=features, filled=True, label='all', rounded=True)
#
#plt.show()

### [Task 4a] (10 points) Answer some questions about the structure of our tree (dtmodel)

#### 1. Can the tree be visualized easily?
#### 2. How deep is the tree?
#### 3. How many nodes it contain?
#### 4. How many total splits are there?
#### 5. What is the impurity of the last 2 nodes?
#### Hint: lookup the scikit-learn documentation to know how to manipulate the 'tree_' attribute of Decision Trees.

In [35]:
###* put your answer as comment here *###
#
# 1. Not easily no, the tree is huge!
# 2. depth = 35
# 3. 18157
# 4. 9078. We can reason as follows: a binary tree of n nodes has n-1 edges, so it must have (n-1)/2 splits (2 edges per split).
# 5. 0
#
nodes = dtmodel.tree_.node_count
nodes, dtmodel.tree_.max_depth, (nodes-1)/2, dtmodel.tree_.impurity[dtmodel.tree_.node_count-1:]

(18157, 35, 9078.0, array([ 0.000]))

#### Let's evaluate the decision tree model.

In [36]:
r2_mse_mae_eval(dtmodel)

Train R^2: 1.000, Val  R^2: 0.968
Train MSE: 0.000, Val MSE: 1076.791
Train MAE: 0.000, Val MAE: 10.691


### [Task 4b] (5 points) Is it a good model? Is it overfitted? Is it better than the models trained in Tasks 2 and 3? (A few sentences suffice.)

In [37]:
###* put your answer as comment here *###
#
# It's a much better model in terms of R^2, MSE, MAE on the validation data compared to the other models from Tasks 2 & 3.
# It's also a very overfitted model as we can see by the fact that the training MSE/MAE are both 0.
#

### [Task 4c] (5 points) Train another decision tree but this time regularize it. Can you obtain a model with similar performance to 'dtmodel' but not (or at least less) overfitted? 

In [38]:
### Call your new model 'dtregmodel'
###* put your code here *###

dtregmodel = DecisionTreeRegressor(random_state=seed, max_depth=20, min_samples_split=12).fit(train_x, train_y)


r2_mse_mae_eval(dtregmodel)

Train R^2: 0.990, Val  R^2: 0.966
Train MSE: 318.945, Val MSE: 1120.187
Train MAE: 6.102, Val MAE: 11.059


### [Task 4d] (5 points) Now let's train a random forest and see if we can train an even better model. Use search_train_eval() to do a grid search over hyperparameters. You are free to pick whatever hyperparameters & values you want, but you should try to avoid badly overfitting.

In [39]:
from sklearn.ensemble import RandomForestRegressor
### Name your random forest model 'rfmodel'
### Make sure to set random_state=seed for reproducibility!
###* put your code here *###

param_grid = {'max_depth':[10, 20, 50, 100], 'min_samples_split': [5, 10, 50]}
rfmodel = search_train_eval(RandomForestRegressor(random_state=seed), param_grid)

RandomForestRegressor({'max_depth': 50, 'min_samples_split': 5})
	Train R^2: 0.996, Val  R^2: 0.983
	Train MSE: 141.907, Val MSE: 578.337
	Train MAE: 3.639, Val MAE: 7.728


### [Task 4e] (5 points) Is your model better than the decision tree you trained for Task 4c? Justify your answer. What do you conclude about ensembles/random forests?

In [40]:
###* put your answer as comment here *###
#
# It's a better model with lower validation error, but it does still overfit significantly.
# This result suggests that ensemble techniques (specifically random forests) 
# can produce overall better models than a single model (of the type used for the weak learners) .
#

## [CIS6930 Additional Task -- Task 5] (25 points): Stacking Meta Model

### For this task we'll use stacking to create a meta model or blender model to predict the target using predictions from 6 other models from Tasks 1 - 4 as features!

### [Task 5a] (10 points) Fill in the code below.

In [41]:
from sklearn.preprocessing import StandardScaler
    
# these are the models we'll use from previous tasks
regressors = [('lr', lrmodel), ('elasticnet', enmodel), ('ridge', rmodel), 
             ('dt', dtmodel), ('dtreg', dtregmodel), ('rf', rfmodel)]

def regressors_preds(x):
    num_regs = len(regressors)
    
    ### Create an array to contain the predictions from the regressors over all examples in 'x'
    ### Each regressor will correspond to one feature (i.e., one column)
    ### The numpy array you return should have shape (x.shape[0], num_regs)
    ###* put your code here (~4-6 lines) *###
    res = np.zeros((x.shape[0],num_regs))
    
    for i in range(0, num_regs):
        name, regr = regressors[i]
        
        pred_reg = regr.predict(x)
        res[:,i] = pred_reg
        
    return res


def stacking_train_eval(model_name, model, standardize=False):
    ### Create a new training dataset meta_train_x and meta_train_y
    ### For this use the validation data (val_x, val_y) alongside with regressors_preds()
    ###* put your code here (~2 lines) *###
    meta_train_x = regressors_preds(val_x)
    meta_train_y = val_y
    
    assert meta_train_x.shape == (2606, 6) and meta_train_x.shape[0] == meta_train_y.shape[0]

    ### Create our new test dataset meta_test_x and meta_test_y
    ### For this use the test data (test_x, test_y) alongside with regressors_preds()
    ###* put your code here (~2 lines) *###
    meta_test_x = regressors_preds(test_x)
    meta_test_y = test_y
    
    assert meta_test_x.shape == (2607, 6) and meta_test_x.shape[0] == meta_test_y.shape[0]
    
    if standardize:
        scaler = StandardScaler()
        meta_train_x = scaler.fit_transform(meta_train_x)
        meta_test_x = scaler.transform(meta_test_x)

    # train the model
    model.fit(meta_train_x, meta_train_y)

    # make predictions & eval
    train_pred = model.predict(meta_train_x)
    test_pred = model.predict(meta_test_x)

    train_error = mean_squared_error(train_pred, meta_train_y)
    val_error = mean_squared_error(test_pred, test_y)

    print('Stacking (Meta model: {})'.format(model_name))
    print('\tTrain MSE: {:.3f}, Test MSE: {:.3f}'.format(train_error, val_error))

    train_error = mean_absolute_error(train_pred, meta_train_y)
    val_error = mean_absolute_error(test_pred, test_y)

    print('\tTrain MAE: {:.3f}, Test MAE: {:.3f}'.format(train_error, val_error))

### [Task 5b] (3 points) Train a SVM regression model with a linear kernel and C=100. Use stacking_train_eval(). You can set standardize=True to zscore normalize features.)

In [42]:
### Train a SVM regressor with a linear kernel and C=100
### Note: the training will take a few minutes
###* put your code here (~2 lines) *###
from sklearn.svm import SVR

svm = SVR(kernel='linear', C=100)
stacking_train_eval('SVM(linear kernel, C=100)', svm, standardize=True)

# note: not rescaling the data makes it take longer but the final model is similar
#stacking_train_eval('SVM(linear kernel, C=100)', svm, standardize=False)

Stacking (Meta model: SVM(linear kernel, C=100))
	Train MSE: 559.621, Test MSE: 722.716
	Train MAE: 7.665, Test MAE: 7.823


### [Task 5c] (2 points) How good is this model? (A few sentences suffice.)

In [43]:
###* put your answer as comment here *###
#
# The model is not as good as the random forests model (in terms of prediction on test data)
# but it does not overfit as much!
#

### [Task 5d] (5 points) Now train a SVM regression model with any other kernel (i.e., not linear) and combination of hyperparameters of your choice. Can you train a better stacking model than for Task 5b?

In [44]:
### Train a SVM regressor with any non-linear kernel and hyperparameters you want
###* put your code here (~2 lines) *###
for kernel in ['rbf', 'poly', 'sigmoid']:
    for C in [1.0, 10, 100, 1000, 10000]:
        svm = SVR(kernel=kernel, C=C)

        stacking_train_eval('SVM({}, C={})'.format(kernel, C), svm, standardize=True)

Stacking (Meta model: SVM(rbf, C=1.0))
	Train MSE: 6431.484, Test MSE: 6417.310
	Train MAE: 28.326, Test MAE: 28.700
Stacking (Meta model: SVM(rbf, C=10))
	Train MSE: 1126.260, Test MSE: 1308.316
	Train MAE: 10.717, Test MAE: 11.367
Stacking (Meta model: SVM(rbf, C=100))
	Train MSE: 567.578, Test MSE: 813.794
	Train MAE: 7.577, Test MAE: 8.297
Stacking (Meta model: SVM(rbf, C=1000))
	Train MSE: 458.209, Test MSE: 804.324
	Train MAE: 6.970, Test MAE: 7.904
Stacking (Meta model: SVM(rbf, C=10000))
	Train MSE: 427.715, Test MSE: 853.571
	Train MAE: 6.485, Test MAE: 8.083
Stacking (Meta model: SVM(poly, C=1.0))
	Train MSE: 18487.651, Test MSE: 17543.485
	Train MAE: 101.286, Test MAE: 98.914
Stacking (Meta model: SVM(poly, C=10))
	Train MSE: 19486.855, Test MSE: 19704.250
	Train MAE: 98.417, Test MAE: 97.711
Stacking (Meta model: SVM(poly, C=100))
	Train MSE: 19282.481, Test MSE: 20731.396
	Train MAE: 97.447, Test MAE: 97.956
Stacking (Meta model: SVM(poly, C=1000))
	Train MSE: 18753.085, T

### [Task 5e] (5 points) What do you conclude? Provide a plausible explanation why non-linear kernels do not seem to improve the result.

In [45]:
###* put your answer as comment here *###
#
# We conclude that stacking can work. In this case though stacking does not provide a significantly better model
# in terms of validation/test data compared to the RF model we trained in Task 4.
#
# A plausible explanation is that since the features are predictions from regressors, 
# the best way to combine them is through a linear combination. This would explain why kernels such RBF have poor performance.
#